In [1]:
#Importing the required libraries
import os
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import random
random.seed(7)
import cv2 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow.keras as keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.utils import np_utils
import time

start = time.time()
#Required Hyperparameter
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 1
NUM_FEATURES = 2048

#Reading the videos
dataset_path = "D:/CSE(AI)/SEM-7/Mini-Project/Data/Videos/"
def load_video(path,resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
    finally:
        cap.release()
    return np.array(frames)
videos_train = []
labels_train = []

test_ind = [21, 61, 65, 43, 55, 22, 50, 35, 50, 33, 64, 64, 82, 23, 31, 60, 1, 3]
videos_test = []
labels_test = []
cntr = 0
for folder in os.listdir(dataset_path):
  for f in os.listdir(dataset_path+folder):
    video_path = dataset_path+folder+"/"+f
    if(cntr in test_ind):
        videos_test.append(load_video(video_path))
        labels_test.append(folder)
    else:
        videos_train.append(load_video(video_path))
        labels_train.append(folder)
    cntr += 1

print("Shape of Single Video",np.shape(videos_train[0]))    

le = LabelEncoder()
for i in range(len(labels_train)):
    if(labels_train[i]):
        labels_train[i] = 0
    else:
        labels_train[i] = 1
y_transform = le.fit_transform(labels_train)
y_cat_train = np_utils.to_categorical(y_transform)

for i in range(len(labels_test)):
    if(labels_test[i]):
        labels_test[i] = 0
    else:
        labels_test[i] = 1
y_transform = le.fit_transform(labels_test)
y_cat_test = np_utils.to_categorical(y_transform)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

#feature_extraction
def build_feature_extractor():
    feature_extractor = keras.applications.inception_v3.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="max",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")
    
feature_extractor = build_feature_extractor()
vid_features_train = []
n_frames_train = []
for i in videos_train:
    temp=feature_extractor.predict(i)
    n_frames_train.append(len(temp))
    vid_features_train.append(temp)
    print(np.shape(temp))

vid_features_test = []
n_frames_test = []
for i in videos_test:
    temp=feature_extractor.predict(i)
    n_frames_test.append(len(temp))
    vid_features_test.append(temp)
    print(np.shape(temp))
  
print("Shape of Video Features",np.shape(vid_features_train))
print("Maximum number of Frames",max(n_frames_train))
print("Average Number of Frames",np.mean(n_frames_train))

print("Shape of Video Features for Test data:",np.shape(vid_features_test))
print("Maximum number of Frames for Test data:",max(n_frames_test))
print("Average Number of Frames for Test data:",np.mean(n_frames_test))

np.save("no_pad_data_train.npy",np.array(vid_features_train))
np.save("no_pad_labels_train.npy",np.array(y_cat_train))
np.save("no_pad_data_test.npy",np.array(vid_features_test))
np.save("no_pad_labels_test.npy",np.array(y_cat_test))

print("<=========================END OF DATA PREPROCESSING (Total Time Taken: {} Minutes) =================================>".format((time.time()-start)/60))

Shape of Single Video (1706, 224, 224, 3)
{0: 0}
(1706, 2048)
(745, 2048)
(909, 2048)
(1895, 2048)
(1745, 2048)
(2203, 2048)
(1664, 2048)
(1846, 2048)
(1790, 2048)
(1568, 2048)
(1575, 2048)
(1586, 2048)
(1368, 2048)
(1564, 2048)
(1801, 2048)
(1683, 2048)
(1630, 2048)
(1711, 2048)
(2113, 2048)
(1576, 2048)
(1530, 2048)
(2004, 2048)
(1479, 2048)
(1161, 2048)
(2126, 2048)
(1507, 2048)
(2186, 2048)
(1570, 2048)
(1642, 2048)
(1607, 2048)
(1383, 2048)
(1838, 2048)
(1739, 2048)
(1056, 2048)
(1627, 2048)
(1520, 2048)
(809, 2048)
(762, 2048)
(1576, 2048)
(873, 2048)
(1216, 2048)
(1047, 2048)
(835, 2048)
(1505, 2048)
(626, 2048)
(1728, 2048)
(1476, 2048)
(1805, 2048)
(1063, 2048)
(774, 2048)
(1448, 2048)
(1296, 2048)
(1416, 2048)
(900, 2048)
(1718, 2048)
(900, 2048)
(900, 2048)
(1135, 2048)
(1233, 2048)
(897, 2048)
(1499, 2048)
(1417, 2048)
(1516, 2048)
(1556, 2048)
(1037, 2048)
(1366, 2048)
(1218, 2048)
(700, 2048)
(1513, 2048)
(1197, 2048)
(1336, 2048)
(1115, 2048)
(638, 2048)
(1700, 2048)
(25